In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
import re

In [3]:
%matplotlib

Using matplotlib backend: MacOSX


In [4]:
# Load files
def load_file(path):
    return pd.read_csv(path, sep=";", header=0, infer_datetime_format=True, parse_dates=['timestamp'], index_col=['timestamp'])

In [5]:
# Define dataset to import
ignore_ids = [223, 45, 19, 105, 75, 63, 58, 59]
directory = "../data/processed/batiments/"
def get_selected_ids(selection_):
    # Get ids
    meta = pd.read_csv(directory+"metadata.csv", sep=';')
    # Use selection
    for select_ in selection_:
        if select_.startswith("max_"):
            if selection_[select_] is not None:
                col = select_[4:]
                meta = meta[meta[col] <= selection_[select_]]
        elif select_.startswith("min_"):
            if selection_[select_] is not None:
                col = select_[4:]
                meta = meta[meta[col] >= selection_[select_]]
        elif select_.startswith("in_"):
            if selection_[select_]:
                col = select_[3:]
                meta = meta[meta[col].isin(selection_[select_])]
        else:
            col = select_
            meta = meta[meta[col] == selection_[col]]
    # Remove manual ids
    meta = meta[~meta["bat_id"].isin(ignore_ids)]
    ids_ = meta["bat_id"].values
    return ids_

def get_list_of_datasets(ids_):
    list_of_datasets_ = []
    list_of_ids_ = []
    for filename_ in os.listdir(directory):
        if re.fullmatch("^[0-9]+.csv", filename_) and int(filename_[:filename_.find('.')]) in ids_:
            list_of_datasets_.append(load_file(directory+filename_))
            list_of_ids_.append(int(filename_[:filename_.find('.')]))
    return list_of_datasets_, list_of_ids_

In [6]:
# name of the column with the value to match or put min_ or max_ before the column name to make a condition
selection = {'time_step': 60,
             'min_bat_id': None,
             'max_bat_id': None,
             'in_bat_id': [53]}
ids = get_selected_ids(selection)
list_of_datasets, list_of_ids = get_list_of_datasets(ids)


print(len(list_of_datasets))
for dataset, id_, real_id_ in zip(list_of_datasets, ids, list_of_ids):
    plt.figure(str(id_) + " - " + str(real_id_))
    plt.plot(dataset.index, dataset['active_power'].values)

1
